<a href="https://colab.research.google.com/github/doma27/phonegap/blob/master/templates/gemini_pro_prompt_CoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [4]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [5]:
model = 'gemini-1.5-pro-002' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IldoZW4gaXMgdGhlIG5leHQgdG90YWwgc29sYXIgZWNsaXBzZSwgYW5kIHdoZXJlIGlzIHRoZSBiZXN0IGxvY2F0aW9uIGZvciB2aWV3aW5nPyJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MC41LCJ0b3BfcCI6MC45NSwidG9wX2siOjQwLCJtYXhfb3V0cHV0X3Rva2VucyI6NDA5Nn0=' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [6]:
gais_contents

[{'role': 'user',
  'parts': [{'text': 'When is the next total solar eclipse, and where is the best location for viewing?'}]}]

In [7]:
generation_config

{'temperature': 0.5, 'top_p': 0.95, 'top_k': 40, 'max_output_tokens': 4096}

In [8]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [9]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [10]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

When is the next total solar eclipse, and where is the best location for viewing? 

-------------------------------------------------------------------------------- 



## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [11]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [12]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [13]:
contents

[{'role': 'user',
  'parts': [{'text': 'When is the next total solar eclipse, and where is the best location for viewing?'}]}]

## Call `generate_content`

In [14]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [15]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

The next total solar eclipse will be on **April 8, 2024**.

The path of totality will cross North America, passing through Mexico, the United States, and Canada.  While the entire continent will experience a partial eclipse, the *best* locations for viewing the total eclipse are along the path of totality.

Some of the locations expected to have good viewing conditions and accessibility within the path of totality include:

* **Mexico:** Mazatlán, Durango
* **United States:**  Texas (including cities like San Antonio and Austin), Arkansas, Missouri, Illinois, Indiana, Ohio, Pennsylvania, New York, Vermont, New Hampshire, Maine
* **Canada:** Southern Ontario, Quebec, New Brunswick, Prince Edward Island, Newfoundland and Labrador

The specific best location for *you* will depend on factors like weather prospects, accessibility, and your personal preferences.  It's recommended to research specific cities and towns within the path of totality closer to the date to get updated weather forecasts and make travel plans.  Remember to book accommodations and transportation well in advance, as these areas will be very popular.


In [16]:
response.candidates

[content {
  parts {
    text: "The next total solar eclipse will be on **April 8, 2024**.\n\nThe path of totality will cross North America, passing through Mexico, the United States, and Canada.  While the entire continent will experience a partial eclipse, the *best* locations for viewing the total eclipse are along the path of totality.\n\nSome of the locations expected to have good viewing conditions and accessibility within the path of totality include:\n\n* **Mexico:** Mazatl\303\241n, Durango\n* **United States:**  Texas (including cities like San Antonio and Austin), Arkansas, Missouri, Illinois, Indiana, Ohio, Pennsylvania, New York, Vermont, New Hampshire, Maine\n* **Canada:** Southern Ontario, Quebec, New Brunswick, Prince Edward Island, Newfoundland and Labrador\n\nThe specific best location for *you* will depend on factors like weather prospects, accessibility, and your personal preferences.  It\'s recommended to research specific cities and towns within the path of totali

## Or Create a chat

In [17]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [18]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

The next total solar eclipse will be on **April 8, 2024**.

The path of totality will arc across North America, passing through Mexico, the United States, and Canada.  While many locations along this path will offer good viewing opportunities, some areas are generally considered better than others due to factors like predicted weather and accessibility.

Some of the locations often cited as ideal for viewing the 2024 eclipse include:

* **Mexico:**  Mazatlán, Durango, and Torreón are expected to have good weather prospects.
* **United States:**  Texas cities like San Antonio, Austin, and Dallas are in the path, as are Indianapolis, Indiana, and Cleveland, Ohio.  Parts of Arkansas are also predicted to have favorable viewing conditions.
* **Canada:**  Southern Ontario, including cities like Hamilton and Niagara Falls, and parts of Quebec and Atlantic Canada will experience the eclipse.

It's important to remember that weather is a crucial factor and can change quickly.  As the date approaches, it's essential to check updated weather forecasts for your chosen viewing location.

Beyond just being in the path of totality, consider factors like:

* **Accessibility:**  How easy is it to get to your chosen location, and how crowded is it likely to be?
* **Local amenities:** Are there facilities like restrooms, food vendors, and parking available?
* **Elevation:** Higher elevations can sometimes offer clearer skies.
* **Length of totality:** The duration of totality varies along the path.  Some locations will experience a longer period of darkness than others.

You can find detailed maps and information about the 2024 eclipse path, including totality duration and local circumstances, on websites like NASA's eclipse website and timeanddate.com.  Start planning your trip early, as accommodations and transportation are likely to book up quickly in popular viewing areas.


In [19]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [20]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))